# Real life pandas work with Kosovo privatization PDFs

## Let's get started!

We're going to work with [example-bid.pdf](example-bid.pdf), a PDF from the [Privatization Agency of Kosovo](https://www.pak-ks.org/page.aspx?id=1%2C106). The agency is responsibile for the privatization of socially owned enterprises, which appears to be done through a competitive bidding process.

The data is *almost* a spreadsheet, but... not exactly. When you have data in a PDF, unfortunately you can't just use `pd.read_pdf` (it doesn't exist!).

You can *try* if you want, but you'll be disappointed.

In [ ]:
import pandas as pd

df = pd.read_pdf("example-bid.pdf")
df.head()

To get the data out of the documents, we're going to use [Natural PDF](https://jsoma.github.io/natural-pdf/), a very nice Python tool that helps you extract information from PDFs. I'm biased because I made it!

We'll start by looking at the **first page.** If we can make it work on the first page, we can make it work on the rest of the pages!

In [ ]:
from natural_pdf import PDF

pdf = PDF("example-bid.pdf")
page = pdf.pages[0]
page.show()

Sometimes you can just ask Natural PDF to give you the entire table. Does that work for us?

In [ ]:
df = page.extract_table().to_df()
df.head()

...not really, the headers are kind of weird and spread out across two rows. And there are also a few empty columns, full of `<NA>` again and again.

Why might that be? Probably something with the headers: **we should look at the PDF to see**. I'll grab the yellowish rectangle and examine it.

In [ ]:
table_header = page.find('rect[fill~=yellow]')
table_header.show(crop=True)

Two things:

1. The horizontal lines are confusing the table extractor
2. The `Tre Ofertuesit me çmim më të lartë` text is getting in the way.

We can fix it most of the way by just removing `Tre Ofertuesit me çmim më të lartë`.

In [ ]:
try:
    # Find the Tre Ofert text and exclude it from the page
    page.find('text:contains(Tre Ofert)').exclude()
except:
    # Already excluded
    pass

# Grab the column names, combine any multi-line ones
column_names = (
    table_header
    .find_all('text')
    .dissolve(vertical=True)
    .extract_each_text(newlines=False, order='ltr')
)
print("Column names are", column_names)

Ok, headers look good, but what about the data? We can then ask for the table from **below the headers**.

In [ ]:
# Take a look at it
table_header.below().show()

In [ ]:
# Now actually grab the data
df = table_header.below().extract_table().to_df(header=None)
df.head()

It's this kind of **multi-step process** that involving thinking and looking that AI isn't very good at yet. It can *kind of* do it *sometimes* but I'd never rely on it (and Natural PDF is too new for it to know much about it).

Our code isn't done yet:

1. We still have those awful `<NA>` columns
2. We don't have the column headers in our dataframe
3. Our column headers repeat for the top 3 bidders
4. If we want to be very picky about our analysis, we can't do math with `€1,111,000`, we need it to be an *actual number*.
5. And this is still only the first page!

While we *could* try to figure this out ourselves, pay attention to what's going on: with the exception of the "do this on every page," **our problems are all pandas work.** It's all manipulating dataframes, something that ChatGPT and friends is very very good at.

And they're also very good at taking a little sample of code and expanding it out over a bigger universe. So let's try that out.

## Step one: Put all your code in one cell

If we're going to ask for help from ChatGPT, we need to put the code all in one place. Run it and make sure it works.

In [ ]:
from natural_pdf import PDF

pdf = PDF("example-bid.pdf")
page = pdf.pages[0]

# Get the table headers

table_header = page.find('rect[fill~=yellow]')

try:
    # Find the Tre Ofert text and exclude it from the page
    table_header.find('text:contains(Tre Ofert)').exclude()
except:
    # Already excluded
    pass

# Grab the column names, combine any multi-line ones
column_names = (
    table_header
    .find_all('text')
    .dissolve(vertical=True)
    .extract_each_text(newlines=False, order='ltr')
)

print("Columns are", column_names)

# Get the table itself
df = table_header.below().extract_table().to_df(header=None)
df.head()

# Talking to ChatGPT

## How to ask for help

A good prompt has five parts:

1. An overall view of what I'm doing (and how)
2. My problem(s), in as much detail as possible.
3. What I want done or changed or fixed. Be specific! You're talking to an *enthusiastic but distractable intern.*
4. My current code
5. The output

## Our prompt

```
In the code below I am scraping a table from the first page of a PDF in a Jupyter notebook. It works, but I need some improvements:

1. The bidding price columns are money with euro and commas symbols in them. Clean them up so I can analyze them.
2. Some of the columns don't have any data in them. Remove those columns (ONLY those NA ones - not zeroes, zeroes are ok)
3. I want the nice column names in the dataframe, but you need to remove the "bad" columns before you assign names
4. The bidder and the bid price column names have duplicate names for the first, second, and third place bidders. Add numbers after the column name to keep it organized.
5. There are multiple pages on the pdf and I want them combined into one dataframe. As you do this I want to see a progress bar.

The code should be "safe" with the data - if anything unexpected happens, provide a warning or an error to show up so we don't lose data. Here is my current code:

from natural_pdf import PDF

pdf = PDF("example-bid.pdf")
page = pdf.pages[0]

table_header = page.find('rect[fill~=yellow]')

try:
    # Find the Tre Ofert text and exclude it from the page
    table_header.find('text:contains(Tre Ofert)').exclude()
except:
    # Already excluded
    pass

column_names = (
    table_header
    .find_all('text')
    .dissolve(vertical=True)
    .extract_each_text(newlines=False, order='ltr')
)

print("Columns are", column_names)

df = table_header.below().extract_table().to_df(header=None)
df.head()

Here is the output:

Columns are ['Nr.', 'Njësitë', 'AKP ID', 'Çmimi më i Lartë', 'Ofertuesi', 'Çmimi', 'Ofertuesi', 'Çmimi', 'Ofertuesi', 'Çmimi']

0	1	2	3	4	5	6	7	8	9	10	11

0	1	Njësia nr.01: Agrokultura Toka në Gjilan (Lot L)	GJI004	€15,127	L106	€15,127	<NA>	0	€0	<NA>	0	€0

1	2	Njësia nr.02: Agrokultura Toka në Gjilan (Lot M)	GJI004	€0	0	€0	<NA>	0	€0	<NA>	0	€0

2	3	Njësia nr.03: Pasuria Bujqësore Toka në Bibaj ...	GJI011	€69,611	L76	€69,611	<NA>	L34	€58,000	<NA>	L63	€12,222

3	4	Njësia nr.04: Pasuria Bujqësore Toka në Ferizaj	GJI011	€1,111,000	L133	€1,111,000	<NA>	L58	€666,666	<NA>	L55	€513,501

4	5	Njësia nr.05: Pasuria Bujqësore Toka në Pojatë...	GJI011	€3,892	L57	€3,892	<NA>	0	€0	<NA>	0	€0
```

Send the code to ChatGPT and you'll (hopefully!) get a working result.

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import warnings
from natural_pdf import PDF

def clean_money(value):
    """Convert euro strings like '€1,111,000' to floats. Handle bad values safely."""
    try:
        if pd.isna(value):
            return pd.NA
        # Remove euro sign and commas, then cast
        return float(str(value).replace("€", "").replace(",", "").strip())
    except Exception as e:
        warnings.warn(f"Could not parse money value '{value}': {e}")
        return pd.NA

# Load PDF
pdf = PDF("example-bid.pdf")

all_dfs = []

for page in tqdm(pdf.pages, desc="Processing pages"):
    # --- Find table header
    table_header = page.find('rect[fill~=yellow]')
    try:
        table_header.find('text:contains(Tre Ofert)').exclude()
    except Exception:
        pass  # already excluded

    # --- Extract raw column names
    column_names = (
        table_header
        .find_all('text')
        .dissolve(vertical=True)
        .extract_each_text(newlines=False, order='ltr')
    )

    # --- Extract table content
    df = table_header.below().extract_table().to_df(header=None)

    # --- Drop fully empty columns BEFORE assigning names
    df = df.dropna(axis=1, how="all")

    # Safety check: column count should match
    if len(df.columns) != len(column_names):
        warnings.warn(
            f"Column name count ({len(column_names)}) != DataFrame cols ({len(df.columns)}). "
            "Truncating to match."
        )
        column_names = column_names[:len(df.columns)]

    # --- Handle duplicate names (Ofertuesi/Çmimi appear multiple times)
    new_columns = []
    seen = {}
    for col in column_names:
        if col in seen:
            seen[col] += 1
            new_columns.append(f"{col}_{seen[col]}")
        else:
            seen[col] = 1
            new_columns.append(f"{col}_1" if col in ["Ofertuesi", "Çmimi"] else col)
    df.columns = new_columns

    # --- Clean up money columns
    for col in df.columns:
        if "Çmimi" in col or "Lartë" in col:
            df[col] = df[col].apply(clean_money)

    all_dfs.append(df)

# --- Combine all pages
final_df = pd.concat(all_dfs, ignore_index=True)

final_df.head()


## Round two: all of the PDFs

Another prompt:

```
I have many pdfs in the folder bid-pdfs/. I would like to similarly read in all of those PDFs and turn them into one big dataframe using the code above. I'd like there to be two new columns, too, 'source_file' and 'page_number'. Save all the errors/warnings up until the end and display them organized by the file name.
```

In [ ]:
import pandas as pd
import warnings
from pathlib import Path
from tqdm.notebook import tqdm
from natural_pdf import PDF

# --- Helper to clean money values
def clean_money(value):
    try:
        if pd.isna(value):
            return pd.NA
        return float(str(value).replace("€", "").replace(",", "").strip())
    except Exception as e:
        return f"PARSE_ERROR: {value} ({e})"  # leave marker instead of silently dropping

# --- Storage
all_dfs = []
error_log = {}

# --- Loop over all PDFs
pdf_files = list(Path("bid-pdfs").glob("*.pdf"))

for pdf_path in tqdm(pdf_files, desc="Processing PDFs"):
    file_errors = []
    try:
        pdf = PDF(str(pdf_path))

        for page_num, page in enumerate(pdf.pages, start=1):
            try:
                # --- Locate header
                table_header = page.find('rect[fill~=yellow]')
                try:
                    table_header.find('text:contains(Tre Ofert)').exclude()
                except Exception:
                    pass  # already excluded

                # --- Extract headers
                column_names = (
                    table_header
                    .find_all('text')
                    .dissolve(vertical=True)
                    .extract_each_text(newlines=False, order='ltr')
                )

                # --- Extract table
                df = table_header.below().extract_table().to_df(header=None)

                # --- Drop fully empty columns BEFORE naming
                df = df.dropna(axis=1, how="all")

                if len(df.columns) != len(column_names):
                    file_errors.append(
                        f"[Page {page_num}] Column mismatch: "
                        f"{len(column_names)} names vs {len(df.columns)} cols"
                    )
                    column_names = column_names[:len(df.columns)]

                # --- Fix duplicate names
                new_columns = []
                seen = {}
                for col in column_names:
                    if col in seen:
                        seen[col] += 1
                        new_columns.append(f"{col}_{seen[col]}")
                    else:
                        seen[col] = 1
                        new_columns.append(f"{col}_1" if col in ["Ofertuesi", "Çmimi"] else col)
                df.columns = new_columns

                # --- Clean money columns
                for col in df.columns:
                    if "Çmimi" in col or "Lartë" in col:
                        df[col] = df[col].apply(clean_money)

                # --- Add metadata columns
                df["source_file"] = pdf_path.name
                df["page_number"] = page_num

                all_dfs.append(df)

            except Exception as e:
                file_errors.append(f"[Page {page_num}] ERROR: {e}")

    except Exception as e:
        file_errors.append(f"[File load error] {e}")

    if file_errors:
        error_log[pdf_path.name] = file_errors

# --- Combine everything
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
else:
    final_df = pd.DataFrame()

# --- Show errors grouped by file
if error_log:
    print("\n===== ERROR / WARNING LOG =====\n")
    for fname, errs in error_log.items():
        print(f"File: {fname}")
        for err in errs:
            print("   -", err)
        print()
else:
    print("No errors or warnings 🎉")

final_df

## Download and analyze

Even if we know what we're looking for, it can always be nice to move over to our friend [ChatGPT](https://chatgpt.com/) to help us analyze the data. Like we saw before: **AI can accurately answer questions with spreadsheets that it can't with PDFs**.

In [ ]:
# or maybe it's called df, who knows!
final_df.to_csv("bids.csv", index=False)

In [ ]:
from IPython.display import FileLink, display

filepath = 'bids.csv'

try:
    # Try to import Colab's download helper
    from google.colab import files
    files.download(filepath)
except ImportError:
    # Fallback: display a clickable link inside the notebook
    display(FileLink(filepath))